In [1]:
import pandas as pd
from jellyfish import jaro_winkler

In [6]:
survey_2016 = "/Users/chris/Code/tempe_survey_data_2016.csv"

In [72]:
df16 = pd.read_csv(survey_2016)

In [8]:
df_describe = df16.describe()

In [9]:
def normalize(col):
    min_val = col["min"]
    norm = 1/(col["max"]-min_val)
    return col.apply(lambda x: (x-min_val)*norm)
df_describe = df_describe.drop("count",axis=0)

In [10]:
df_norm = df_describe.apply(normalize,axis=0)
row_indexes = df_norm.index

In [11]:
def get_distance(col1,col2):
    return sum(abs(df_norm[col1][row]-df_norm[col2][row]) for row in row_indexes)

In [12]:
from itertools import combinations
def get_top_k_features(k=5):
    cols_distances = {}
    key_pairs = combinations(df_norm.keys(),2)
    for cols in key_pairs:
        key = tuple(sorted(cols))
        if key not in cols_distances:
            cols_distances[key] = get_distance(key[0], key[1])
    return sorted(cols_distances.items(), key=lambda x: x[1])[:k]

In [13]:
get_top_k_features()

[(('Q34  I believe exceptional job performanc',
   'Q67  How has the quality of the work envi'),
  0.0014203349949593047),
 (('Q27  Within the last 30 days I have recei',
   'Q56  I am encouraged to be innovative and'),
  0.001515698196722981),
 (('Q20  My work is appreciated by my immedia',
   'Q62  Overall  I am very satisfied with th'),
  0.001522062099594712),
 (('Q50  I can express my opinions about work',
   'Q56  I am encouraged to be innovative and'),
  0.001757359557513244),
 (('Q27  Within the last 30 days I have recei',
   'Q50  I can express my opinions about work'),
  0.0020947872373406062)]

In [39]:
a = jaro_winkler(full_col_names[0], full_col_names[1])

<timed exec>:2: DeprecationWarning: PY_SSIZE_T_CLEAN will be required for '#' formats


CPU times: user 404 ms, sys: 5.26 ms, total: 410 ms
Wall time: 412 ms


In [51]:
def get_full_col_name():
    pass

def autocomplete_col_names():
    pass

In [85]:
import re

df16 = pd.read_csv(survey_2016)
full_col_names_file = "/Users/chris/Code/TempeSurveyReport/full_questions_2016.txt"
full_col_names = []
with open(full_col_names_file, "r") as f:
    full_col_names = f.read().split("\n")

full_name_question_map = {}
special_questions = {
    "9": " The following adequately support my work-related needs:",
    "10": " The following programs/services adequately support my needs:",
}

for text in full_col_names:
    period_loc = text.find(".")
    question_number = text[:period_loc]
    if question_number:
        stripped_number = re.sub("[^0-9]", "", question_number)
        if stripped_number in special_questions:
            special_text = (
                "Q"
                + question_number
                + "."
                + special_questions[stripped_number]
                + text[period_loc + 1 :]
            )
            full_name_question_map["Q" + question_number] = special_text
        else:
            full_name_question_map["Q" + question_number] = "Q" + text

In [89]:
print(full_name_question_map)

{'Q1': 'Q1. I receive training to do my job effectively', 'Q2': 'Q2. There is someone at work who encourages my development', 'Q3': 'Q3. I have been mentored at work', 'Q4': 'Q4. I have received fair consideration for advancement and promotion, when available, within the City of Tempe', 'Q5': "Q5. I am aware of the City's educational partnerships, Tempe Professional Development Network, The Mentoring Program, and other programs related to professional development and career mobility", 'Q6': "Q6. The City's programs related to professional development and career mobility, such as educational partnerships, Tempe Professional Development Network, etc., are useful to me", 'Q7': 'Q7. Overall, I am satisfied with the professional development opportunities that are available to me at the City', 'Q8': 'Q8. How could the City improve professional development and/or career mobility for employees?', 'Q9': 'Q9. The following adequately support my work-related needs: The following adequately suppor

In [86]:
rename_dict = {}
for key in df16.keys():
    rename_dict[key] = key.split(" ")[0]
df16 = df16.rename(columns=rename_dict)
section_keys = df16.keys()
def get_section()

In [87]:
df16.keys()

Index(['ID', 'Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q9a', 'Q9b', 'Q9c',
       'Q9d', 'Q9e', 'Q10a', 'Q10b', 'Q10c', 'Q11', 'Q12', 'Q13', 'Q15', 'Q16',
       'Q17', 'Q18', 'Q19', 'Q20', 'Q21', 'Q22', 'Q23', 'Q24', 'Q25', 'Q26',
       'Q27', 'Q28', 'Q29', 'Q30', 'Q31', 'Q32', 'Q33', 'Q34', 'Q35', 'Q36',
       'Q37', 'Q38', 'Q39', 'Q41', 'Q42', 'Q43', 'Q44', 'Q45', 'Q46', 'Q47',
       'Q49', 'Q50', 'Q51', 'Q52', 'Q53', 'Q54', 'Q55', 'Q56', 'Q57', 'Q58',
       'Q60', 'Q61', 'Q62', 'Q64', 'Q65', 'Q66', 'Q67', 'Q68', 'Q69', 'Q70',
       'Q71', 'Q74'],
      dtype='object')

In [44]:
key_map = {}
current_keys = set(df16.keys())-set(["ID"])
for col in df16.keys():
    top_sim = 0
    best_key = ""
    for match_col in full_col_names:
        sim = jaro_winkler(col,match_col)
        if sim > top_sim:
            top_sim = sim
            best_key = match_col
    key_map[col] = best_key

<ipython-input-44-d3a95ec78f61>:7: DeprecationWarning: PY_SSIZE_T_CLEAN will be required for '#' formats
  sim = jaro_winkler(col,match_col)


In [45]:
key_map

{'ID': "5. I am aware of the City's educational partnerships, Tempe Professional Development Network, The Mentoring Program, and other programs related to professional development and career mobility",
 'Q1  I receive training to do my job effec': '1. I receive training to do my job effectively',
 'Q2  There is someone at work who encourag': '2. There is someone at work who encourages my development',
 'Q3  I have been mentored at work': '3. I have been mentored at work',
 'Q4  I have received fair consideration fo': '54. I have adequate input on decisions affecting my work',
 "Q5  I am aware of the City's educational": '37. I feel safe and comfortable in my work unit',
 "Q6  The City's programs related to profes": '63. How could the City improve peer relationships among employees?',
 'Q7  Overall  I am satisfied with the prof': '38. My immediate supervisor has the tools to deal with conflict',
 "Q9a  City Manager's Office": "9a. City Manager's Office",
 'Q9b  Strategic Management & Di